In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# ignore possible warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
movies = pd.read_csv('datasets/movies.csv')
credits = pd.read_csv('datasets/credits.csv')

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
credits.head(1)['cast'].values

credits.head(1)['crew'].values

In [ ]:
movies_db = movies.merge(credits, on = 'title')

In [ ]:
movies_db.head(1)

In [ ]:
movies_db.columns

In [ ]:
# we are keeping the following:

# genres
# id
# keywords
# overview
# title
# cast
# crew

movies_final_db = movies_db[['id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies_final_db.head(1)

In [ ]:
movies_final_db.isnull().sum()

In [ ]:
#drop rows with null 
movies_final_db.dropna(inplace=True)

In [ ]:
movies_final_db.isnull().sum()

In [ ]:
movies_final_db.duplicated().sum()

In [ ]:
movies_final_db.iloc[0].genres

In [ ]:
import ast

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L 

In [ ]:
movies_final_db['genres'] = movies_final_db['genres'].apply(convert)
movies_final_db['keywords'] = movies_final_db['keywords'].apply(convert)
movies_final_db['cast'] = movies_final_db['cast'].apply(convert)

In [ ]:
movies_final_db.head(1)

In [ ]:
movies_final_db['crew'][0]

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L 

movies_final_db['crew'] = movies_final_db['crew'].apply(fetch_director)
# movies_final_db['crew'].apply(fetch_director)

In [ ]:
movies_final_db.head(1)

In [ ]:
#converting the overview (string) into list
movies_final_db['overview'].apply(lambda x : x.split())

In [ ]:
movies_final_db['overview'] = movies_final_db['overview'].apply(lambda x : x.split())

In [ ]:
movies_final_db.head(1)

In [ ]:
movies_final_db['genres'] = movies_final_db['genres'].apply(lambda x:[i.replace(" " , "") for i in x])
movies_final_db['keywords'] = movies_final_db['keywords'].apply(lambda x:[i.replace(" " , "") for i in x])
movies_final_db['cast'] = movies_final_db['cast'].apply(lambda x:[i.replace(" " , "") for i in x])
movies_final_db['crew'] = movies_final_db['crew'].apply(lambda x:[i.replace(" " , "") for i in x])

In [ ]:
movies_final_db.head(1)

In [ ]:
# concate columns (overview, genres,keywords,cast,crew) into new 'tags' column
movies_final_db['tags'] = movies_final_db['overview'] + movies_final_db['genres'] + movies_final_db['keywords'] + movies_final_db['cast'] + movies_final_db['crew']

In [ ]:
movies_final_db.head(1)

In [ ]:
movies_final_db['tags'][0]

### new dataframe

In [ ]:
new_df = movies_final_db[['id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'].apply(lambda x: x.lower())

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [ ]:
new_df.head()

# applying steming

In [ ]:
import nltk

In [ ]:
# !pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_df['tags'].apply(stem)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

### text vectorization scikitlearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(max_features = 6000, stop_words=True)
cv = CountVectorizer(max_features = 5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors.shape
vectors

In [ ]:
#most frequent 5000 words
cv.get_feature_names()
# len(cv.get_feature_names())

### cosine_similarity scikitlearn

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(vectors)

In [ ]:
cosine_similarity(vectors).shape

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
similarity

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    recommended_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6]
    
    for i in recommended_list:
        print(new_df.iloc[i[0]].title)
#         print(i[0])

In [ ]:
recommend('Avatar')
# recommend('Fight Club')

### exporting movies names to website

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
#exporting as a dictionary
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))

In [ ]:
## dumping similarity
pickle.dump(similarity,open('similarity.pkl','wb'))